# QTM 350 - SQL Practice Session
## Analysing Classic Video Games & Studios
**Author:** Danilo Freire (danilo.freire@emory.edu, Emory University)

### Objective
In this session, you will practice your SQL skills by creating and querying a database of classic video games and their development studios. You will use `SELECT` statements, aggregate functions, joins, and other SQL features to answer a series of questions.

### Instructions
1. Run the **Setup** cells to create and populate your database for this session.
2. For each question, write a single SQL query in the designated code cell to find the answer.
3. Use `pd.read_sql()` to execute your query and display the results as a pandas DataFrame.

## Part 1: Database and Table Setup

In [1]:
import sqlite3
import pandas as pd
from IPython.display import display

# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

print("Database connection established.")

Database connection established.


### Table Schemas

We will work with two tables: `studios` and `games`.

**`studios` table:**
| Column         | Type    | Description                               |
|----------------|---------|-------------------------------------------|
| `studio_id`    | INTEGER | Primary Key for the studio.               |
| `studio_name`  | TEXT    | The name of the game development studio.  |
| `country`      | TEXT    | The country where the studio is based.    |
| `year_founded` | INTEGER | The year the studio was founded.          |

**`games` table:**
| Column               | Type    | Description                                      |
|----------------------|---------|--------------------------------------------------|
| `game_id`            | INTEGER | Primary Key for the game.                        |
| `title`              | TEXT    | The title of the video game.                     |
| `studio_id`          | INTEGER | Foreign Key referencing `studios(studio_id)`.      |
| `genre`              | TEXT    | The genre of the game (e.g., RPG, Platformer).   |
| `release_year`       | INTEGER | The year the game was released.                  |
| `units_sold_millions`| REAL    | Units sold, in millions.                         |

In [2]:
# Drop tables if they already exist to ensure a clean slate
cursor.execute("DROP TABLE IF EXISTS games;")
cursor.execute("DROP TABLE IF EXISTS studios;")

# Create studios table
cursor.execute("""
CREATE TABLE studios (
    studio_id INTEGER PRIMARY KEY,
    studio_name TEXT NOT NULL,
    country TEXT,
    year_founded INTEGER
);
""")

# Create games table
cursor.execute("""
CREATE TABLE games (
    game_id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    studio_id INTEGER,
    genre TEXT,
    release_year INTEGER,
    units_sold_millions REAL,
    FOREIGN KEY (studio_id) REFERENCES studios(studio_id)
);
""")

print("Tables 'studios' and 'games' created successfully.")

Tables 'studios' and 'games' created successfully.


In [3]:
# Insert data into studios table
studios_data = [
    (1, 'Nintendo', 'Japan', 1889),
    (2, 'Blizzard Entertainment', 'USA', 1991),
    (3, 'Square Enix', 'Japan', 2003),
    (4, 'CD Projekt Red', 'Poland', 1994),
    (5, 'Rockstar Games', 'USA', 1998),
    (6, 'FromSoftware', 'Japan', 1986)
]
cursor.executemany("INSERT INTO studios VALUES (?, ?, ?, ?)", studios_data)

# Insert data into games table
games_data = [
    (1, 'The Legend of Zelda: Ocarina of Time', 1, 'Action-Adventure', 1998, 7.6),
    (2, 'World of Warcraft', 2, 'MMORPG', 2004, 12.0),
    (3, 'Final Fantasy VII', 3, 'RPG', 1997, 10.0),
    (4, 'The Witcher 3: Wild Hunt', 4, 'RPG', 2015, 50.0),
    (5, 'Super Mario 64', 1, 'Platformer', 1996, 11.9),
    (6, 'Grand Theft Auto V', 5, 'Action-Adventure', 2013, 195.0),
    (7, 'Diablo II', 2, 'Action RPG', 2000, 4.0),
    (8, 'Chrono Trigger', 3, 'RPG', 1995, 2.63),
    (9, 'Elden Ring', 6, 'Action RPG', 2022, 23.0),
    (10, 'Red Dead Redemption 2', 5, 'Action-Adventure', 2018, 61.0)
]
cursor.executemany("INSERT INTO games VALUES (?, ?, ?, ?, ?, ?)", games_data)

conn.commit()
print("Data inserted successfully.")

Data inserted successfully.


## Part 2: Analysis Questions

### Question 1: Basic Filtering
List all games released **after** the year 2010. Show the game `title` and its `release_year`.

In [5]:
# Your SQL query for Question 1 here
query1 = """
SELECT title, release_year FROM games
WHERE release_year > 2010;
"""

display(pd.read_sql(query1, conn))

,title,release_year
0,The Witcher 3: Wild Hunt,2015
1,Grand Theft Auto V,2013
2,Elden Ring,2022
3,Red Dead Redemption 2,2018


### Question 2: Aggregation with `SUM` and `WHERE`
Find the total units sold (in millions) for all games in the 'RPG' genre.

In [21]:
# Your SQL query for Question 2 here
query2 = """
SELECT genre, 
    SUM(units_sold_millions) AS sum_sold
FROM games
WHERE genre IN ('RPG')
"""

display(pd.read_sql(query2, conn))

,genre,sum_sold
0,RPG,62.63


### Question 3: `ORDER BY` and `LIMIT`
Show the name and release year of the top 3 best-selling games (by `units_sold_millions`).

In [24]:
# Your SQL query for Question 3 here
query3 = """
SELECT title, release_year
FROM games
ORDER BY units_sold_millions DESC
LIMIT 3;
"""

display(pd.read_sql(query3, conn))

,title,release_year
0,Grand Theft Auto V,2013
1,Red Dead Redemption 2,2018
2,The Witcher 3: Wild Hunt,2015


### Question 4: `LEFT JOIN`
List all games and their studio names. Ensure that all games are listed, even if their studio is not in the `studios` table (which is not the case in our sample data, but the query should be written to handle it).

In [26]:
# Your SQL query for Question 4 here
query4 = """
SELECT games.title, studios.studio_name
FROM games
LEFT JOIN studios ON games.studio_id = studios.studio_id;
"""

display(pd.read_sql(query4, conn))

,title,studio_name
0,The Legend of Zelda: Ocarina of Time,Nintendo
1,World of Warcraft,Blizzard Entertainment
2,Final Fantasy VII,Square Enix
3,The Witcher 3: Wild Hunt,CD Projekt Red
4,Super Mario 64,Nintendo
5,Grand Theft Auto V,Rockstar Games
6,Diablo II,Blizzard Entertainment
7,Chrono Trigger,Square Enix
8,Elden Ring,FromSoftware
9,Red Dead Redemption 2,Rockstar Games


### Question 5: `GROUP BY` with an Aggregate Function
Calculate the average units sold for each studio. Show the studio name and their average sales, aliased as `avg_sales`. Order the results by average sales in descending order.

In [ ]:
# Your SQL query for Question 5 here
query5 = """
SELECT studios.studio_name, AVG(games.units_sold_millions) AS avg_sales
FROM games
LEFT JOIN studios ON games.studio_id = studios.studio_id
GROUP BY studios.studio_name
"""

display(pd.read_sql(query5, conn))

,studio_name,avg_sales,total_games
0,Blizzard Entertainment,8.000,2
1,CD Projekt Red,50.000,1
2,FromSoftware,23.000,1
3,Nintendo,9.750,2
4,Rockstar Games,128.000,2
5,Square Enix,6.315,2


### Question 6: `GROUP BY` with `HAVING`
Find all studios that have released **more than one** game. Show the studio name and the count of games they have released.

In [39]:
# Your SQL query for Question 6 here
query6 = """
SELECT studios.studio_name, COUNT(games.game_id) AS total_games
FROM studios
LEFT JOIN games ON studios.studio_id = games.studio_id
GROUP BY studios.studio_name
HAVING COUNT(games.game_id) > 1
"""

display(pd.read_sql(query6, conn))

,studio_name,total_games
0,Blizzard Entertainment,2
1,Nintendo,2
2,Rockstar Games,2
3,Square Enix,2


### Question 7: String Manipulation with `LIKE`
Find all game titles that contain the word `The`.

In [42]:
# Your SQL query for Question 7 here
query7 = """
SELECT title FROM games
WHERE title LIKE '%The %'
"""

display(pd.read_sql(query7, conn))

,title
0,The Legend of Zelda: Ocarina of Time
1,The Witcher 3: Wild Hunt


### Question 8: Conditional Logic with `CASE`
Categorise games based on their sales: 'Blockbuster' for games with over 50 million units sold, 'Hit' for games with 10 to 50 million units sold (inclusive), and 'Classic' for games with under 10 million units sold. Show the game `title` and its `sales_category`.

In [44]:
# Your SQL query for Question 8 here
query8 = """
SELECT title,
    CASE
        WHEN units_sold_millions > 50 THEN 'Blockbuster'
        WHEN units_sold_millions BETWEEN 10 AND 50 THEN 'Hit'
        WHEN units_sold_millions < 10 THEN 'Classic'
    END AS sales_category
FROM games
"""

display(pd.read_sql(query8, conn))

,title,sales_category
0,The Legend of Zelda: Ocarina of Time,Classic
1,World of Warcraft,Hit
2,Final Fantasy VII,Hit
3,The Witcher 3: Wild Hunt,Hit
4,Super Mario 64,Hit
5,Grand Theft Auto V,Blockbuster
6,Diablo II,Classic
7,Chrono Trigger,Classic
8,Elden Ring,Hit
9,Red Dead Redemption 2,Blockbuster


### Question 9: Subquery in a `WHERE` Clause
List all games that have sold more units than the average of all games in the database.

In [69]:
# Your SQL query for Question 9 here
query9 = """
SELECT title, units_sold_millions
FROM games
WHERE units_sold_millions > (SELECT AVG(units_sold_millions) FROM games);
"""

display(pd.read_sql(query9, conn))

,title,units_sold_millions
0,The Witcher 3: Wild Hunt,50.0
1,Grand Theft Auto V,195.0
2,Red Dead Redemption 2,61.0


### Question 10: Window Function (`RANK`)
Rank games within each `genre` based on their release year (from oldest to newest). Show the `genre`, game `title`, `release_year`, and the rank.

In [74]:
# Your SQL query for Question 10 here
query10 = """
SELECT title, genre, release_year,
RANK() OVER (ORDER BY release_year ASC) AS rank_overall
FROM games
ORDER BY rank_overall;
"""

display(pd.read_sql(query10, conn))

,title,genre,release_year,rank_overall
0,Chrono Trigger,RPG,1995,1
1,Super Mario 64,Platformer,1996,2
2,Final Fantasy VII,RPG,1997,3
3,The Legend of Zelda: Ocarina of Time,Action-Adventure,1998,4
4,Diablo II,Action RPG,2000,5
5,World of Warcraft,MMORPG,2004,6
6,Grand Theft Auto V,Action-Adventure,2013,7
7,The Witcher 3: Wild Hunt,RPG,2015,8
8,Red Dead Redemption 2,Action-Adventure,2018,9
9,Elden Ring,Action RPG,2022,10


## Part 3: Cleanup

When you are finished, it's good practice to close the database connection.

In [75]:
# Close the connection
conn.close()
print("Database connection closed.")

Database connection closed.
